In [169]:
from collections import defaultdict

### lecture des fichiers

In [170]:
def parse_graph(filename): 
    dico = defaultdict(dict)
    with open(filename) as f:
        for line in f:
            summit1, summit2, poids = line.strip().split(',')
            poids = int(poids)
            if summit1 in dico :
                dico[summit1][summit2] = poids
            else :
                dico[summit1] = {summit2 : poids}
    return dico

G = parse_graph('graph.csv')
G

defaultdict(dict,
            {'a': {'b': 7, 'd': 9, 'c': 14},
             'b': {'d': 10, 'e': 15},
             'c': {'d': 2, 'f': 9},
             'd': {'e': 11},
             'e': {'f': 6}})

In [171]:
def number_vertices(D):
    deja_vu = set()
    """
    returns number of vertices
    
    Parameters:
      graph: implemented as a dictionary of adjacency dictionaries
      
    Returns:
      int: number of vertices
    """
    for key, value in D.items():
        if key not in deja_vu : deja_vu.add(key)
        for key2 in value.keys():
            if key2 not in deja_vu : deja_vu.add(key2)
    return len(deja_vu)
number_vertices(G)

6

In [172]:
def reachables(graph, s):
    """
    computes the set of reachable vertices in a graph from source s

    Parameters:
      graph: a graph implemented as a dict of adjacency dicts
      s: the source vertex
    Returns:
      a set of vertices in graph
    """
    already_seen = set([s])
    available = set(graph[s].keys())
    while not available.issubset(already_seen):
      new_available = set()
      new_seen = set()
      for s2 in available:
        already_seen.add(s2)
        new_seen.add(s2)
        new_available.update(set(graph[s2].keys()))
      available.difference_update(new_seen)
      available.update(new_available)
    return already_seen

reachables(G, 'd')

{'d', 'e', 'f'}

In [173]:
# comment écrire un petit test plus informatif
# sur le premier graphe témoin

# on énumère à la main les sommets à tester 
# et les résultats attendus
G_tests = [
    ('a', {'a', 'b', 'c', 'd', 'e', 'f'}),
    ('b', {'b', 'd', 'e', 'f'}),
    ('c', {'c', 'd', 'e', 'f'}),
    ('d', {'d', 'e', 'f'}),
    ('e', {'e', 'f'}),
    ('f', {'f'}),
]

# on vérifie pour chacun qu'on
# obtient bien le résultat attendu
for (s, expected) in G_tests:
    computed = reachables(G, s)
    if computed != expected:
        print(f"ERROR found {computed} != {expected}")
    else:
        print(f"depuis {s} → {computed}")

depuis a → {'d', 'c', 'e', 'f', 'b', 'a'}
depuis b → {'d', 'b', 'e', 'f'}
depuis c → {'d', 'e', 'c', 'f'}
depuis d → {'d', 'e', 'f'}
depuis e → {'f', 'e'}
depuis f → {'f'}


In [174]:
def shortest_distance1(graph, a, f):
    mark = {a : 0}
    visited = set([a]) #sinon V1 devient {'1', 'V'}
    s0 = a
    while f not in visited :
        
        min_mark = 10000
        d0 = None
        i = 0
        for s in visited:                
                if s in graph : 
                    for d in graph[s].keys():
                        
                        if d not in visited:
                            
                            i += 1
                            m = mark[s] + graph[s][d]
                            if m <= min_mark:
                                min_mark = m
                                d0 = d
                                s0 = s

        if i == 0 : return 'not available'
    
        
        visited.add(d0)
        mark[d0] = mark[s0] + graph[s0][d0]
    return mark[d0]
    
          
shortest_distance1(G, 'a', 'e')            



20

In [175]:
G2 = parse_graph('data/graph2.csv')
G2

defaultdict(dict,
            {'v1': {'v3': 5, 'v2': 1},
             'v2': {'v3': 1, 'v4': 3, 'v5': 4},
             'v3': {'v4': 1},
             'v4': {'v5': 1, 'v6': 3},
             'v5': {'v6': 1}})

In [176]:
shortest_distance1(G2, 'v1', 'v6')

5

In [177]:
def shortest_path(graph, a, f):
    mark = {a : (0,[a])}
    visited = set([a]) #[] sinon V1 devient {'1', 'V'}
    s0 = a
    while f not in visited :
        
        min_mark = 100000
        d0 = 'a'
        i = 0
        for s in visited:                
                for d in graph[s].keys():                        
                    if d not in visited:                            
                        i += 1
                        m = mark[s][0] + graph[s][d]
                        if m <= min_mark:
                            min_mark = m
                            d0 = d
                            s0 = s

        if i == 0 : return 'not available'
    
        
        visited.add(d0)
        mark[d0] =(mark[s0][0] + graph[s0][d0], mark[s0][1] + [d0]) 
    return mark[d0]
shortest_path(G, 'a', 'f')

(23, ['a', 'c', 'f'])

Allons chercher les données :

In [178]:
# voici l'idiome qui permet d'aller chercher 
# une page web à partir de son URL
import requests
thrones_url = "https://raw.githubusercontent.com/pupimvictor/NetworkOfThrones/master/stormofswords.csv"
get_request = requests.get(thrones_url)
text_data = get_request.text


In [179]:
data = text_data.split('\n')[1:-1] #contenait une ligne vide à la fin qui plantait parse_graph
with open('data/my_thrones.csv', 'w') as f :
    for character in data :
        f.write(character + '\n')


In [180]:
!head -5 data/my_thrones.csv

Aemon,Grenn,5
Aemon,Samwell,31
Aerys,Jaime,18
Aerys,Robert,6
Aerys,Tyrion,5


In [181]:
thrones =  parse_graph('data/my_thrones.csv')


In [182]:
print(f'{number_vertices(thrones)=}')
print(len(reachables(thrones, 'Shireen')))
shortest_path(thrones, 'Margaery', 'Eddard')

number_vertices(thrones)=107
4


'not available'